### Simple explanation of descriptors (描述符)

简单来说，描述符就是绑定有特定行为的对象属性，其属性访问（attribute access）会被描述符协议中的方法重写。这些方法是`__get__()`，`__set__()`，`__delete__()`。
- `obj.attribute` --> `attribute.__get__(obj, type(obj))`
- `obj.attribute = value` --> `attribute.__set__(obj, value)`
- `del obj.attribute` --> `attribute.__delete__(obj)`

当使用`.`来访问函数（方法）的时候，其实是调用了`__get__()`方法来将函数绑定给类的实例，使其成为这个对象的bound method。

In [4]:
def f(obj): pass

hasattr(f, '__get__')

True

In [5]:
f

<function __main__.f(obj)>

In [7]:
obj = object()

In [8]:
f.__get__(obj, type(obj))

<bound method f of <object object at 0x7f4150917190>>

所以，当调用一个类的方法时，并不是调用了原函数对象的`__call__()`，而是调用了由于访问此函数而产生的临时绑定对象（bound object）的`__call__()`方法。

In [9]:
class Object(object):
    def f(self): pass

In [10]:
obj = Object()
obj.f

<bound method Object.f of <__main__.Object object at 0x7f414c1f1978>>

回顾之前的错误：

In [14]:
import functools 

class function_wrapper(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped
        functools.update_wrapper(self, wrapped)
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

class Class(object):
    @function_wrapper
    @classmethod
    def cmethod(cls):
        pass 

Class.cmethod() 

TypeError: 'classmethod' object is not callable

这是因为若要`@classmethod`装饰器正确工作，必须正确地应用描述符协议（descriptor protocol）。`__call__()`方法是通过`__get__()`来返回的，而`@classmethod`自身并没有`__call__()`方法。更明确地说，因为简单版本的装饰器并不遵守描述符协议来产生一个能被正确调用的绑定函数，而是简单地调用被包装的对象，当此对象没有`__call__()`方法时就会失败。

当把其应用到普通的实例方法时，可以正常工作，因为普通的函数有`__call__()`方法，虽然绑定协议被放在一旁，但当调用原来的未绑定的函数对象时，装饰器显式地传递了`self`参数。

### Wrappers as descriptors

解决方案：同时把装饰器实现为描述符

查看`update_wrapper`在Python 3.4+中的实现：

```python
WRAPPER_ASSIGNMENTS = ('__module__', '__name__', '__qualname__', '__doc__',
                       '__annotations__')
WRAPPER_UPDATES = ('__dict__',)
def update_wrapper(wrapper,
                   wrapped,
                   assigned = WRAPPER_ASSIGNMENTS,
                   updated = WRAPPER_UPDATES):
    """Update a wrapper function to look like the wrapped function

       wrapper is the function to be updated
       wrapped is the original function
       assigned is a tuple naming the attributes assigned directly
       from the wrapped function to the wrapper function (defaults to
       functools.WRAPPER_ASSIGNMENTS)
       updated is a tuple naming the attributes of the wrapper that
       are updated with the corresponding attribute from the wrapped
       function (defaults to functools.WRAPPER_UPDATES)
    """
    for attr in assigned:
        try:
            value = getattr(wrapped, attr)
        except AttributeError:
            pass
        else:
            setattr(wrapper, attr, value)
    for attr in updated:
        getattr(wrapper, attr).update(getattr(wrapped, attr, {}))
    # Issue #17482: set __wrapped__ last so we don't inadvertently copy it
    # from the wrapped function when updating __dict__
    wrapper.__wrapped__ = wrapped
    # Return the wrapper so this can be used as a decorator via partial()
    return wrapper
```

其分别复制了被包装函数的各类属性以及`__dict__`的内容，最后将函数绑定给`__wrapped__`

In [74]:
import functools 

class bound_function_wrapper(object): 
    def __init__(self, wrapped):
        self.wrapped = wrapped 
        functools.update_wrapper(self, wrapped)
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs) 

class function_wrapper(object): 
    def __init__(self, wrapped):
        self.wrapped = wrapped 
        functools.update_wrapper(self, wrapped) # Caution!
    def __get__(self, instance, owner):
        wrapped = self.wrapped.__get__(instance, owner)
        return bound_function_wrapper(wrapped) 
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

In [75]:
@function_wrapper
def func(): pass

f = func
getattr(f, '__dict__')

{'wrapped': <function __main__.func()>,
 '__module__': '__main__',
 '__name__': 'func',
 '__qualname__': 'func',
 '__doc__': None,
 '__annotations__': {},
 '__wrapped__': <function __main__.func()>}

In [77]:
# Check the getattr function
getattr.__doc__

"getattr(object, name[, default]) -> value\n\nGet a named attribute from an object; getattr(x, 'y') is equivalent to x.y.\nWhen a default argument is given, it is returned when the attribute doesn't\nexist; without it, an exception is raised in that case."

每次将函数绑定给某个类时都需要创建一个bound wrapper，性能不够好，使用transparent object proxy来解决这个问题。

Example(实际的实现要复杂的多):

In [79]:
class object_proxy(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped
        try:
            self.__name__ = wrapped.__name__
        except AttributeError:
            pass
        
    @property
    def __class__(self):
        return self.wrapped.__class__
    
    def __getattr__(self, name):
        return getattr(self.wrapped, name)

In [83]:
class bound_function_wrapper(object_proxy):
    def __init__(self, wrapped):
        super(bound_function_wrapper, self).__init__(wrapped)
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)  

class function_wrapper(object_proxy):
    def __init__(self, wrapped):
        super(function_wrapper, self).__init__(wrapped)
    def __get__(self, instance, owner):
        wrapped = self.wrapped.__get__(instance, owner)
        return bound_function_wrapper(wrapped) 
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs) 

这样方式的实现同时能让`inspect.getargspec()`和`inspect.getsource()`正常工作